In [1]:
import geopandas as gpd
import rasterio
import rasterio.mask
import geopandas as gpd
import numpy as np
import pandas as pd
import glob as glob
import rioxarray
import os
import gc
    
from osgeo import gdal
from osgeo import ogr
from shapely import wkb
from shapely.geometry import Polygon, MultiPolygon


from sklearn.cluster import MiniBatchKMeans
from yellowbrick.cluster import KElbowVisualizer
from sklearn.model_selection import train_test_split


In [ ]:
# Specify paths
rasPath = "D:\\TEMP\\_output\\_tif\\_Manning_Invasives\\Struc_Spec_1cm_near_fill.tif"
rasQuadratPath = 'E:\\Sync\\_Documents\\_Letter_invasives\\_Data\\_quadrats\\'


In [ ]:
# Read Vector data

# Read Minority training data
gdfMinority = gpd.read_file("E:\\Sync\\_Documents\\_Letter_invasives\\_Data\\Training_samples_both_species2.shp").set_crs('EPSG:26911')

# Read Transect outlines
quadrats = gpd.read_file('E:\Sync\_Documents\_Letter_invasives\_Data\Transect_grids.shp').set_crs('EPSG:26911')

### Raster within quadrats to points (GeoDataFrame)
Using GDAL Warp (clipping) and Rioxarray (conversion)

In [5]:
# Split quadrats by transect and write as seperate shapefiles

quadratsSplit = [gpd.GeoSeries((quadrats[quadrats['Quadrat'].isin(['1_2', '1_3', '1_1'])])['geometry'].unary_union),
                 gpd.GeoSeries((quadrats[quadrats['Quadrat'].isin(['2_2', '2_7', '2_5', '2_9', '2_4', '2_10', '2_1', '2_3', '2_6', '2_8'])])['geometry'].unary_union),
                 gpd.GeoSeries((quadrats[quadrats['Quadrat'].isin(['3_3', '3_2', '3_1'])])['geometry'].unary_union)]

for i in range(3):
    j = str(i+1)
    outPath = f'E:\\Sync\\_Documents\\_Letter_invasives\\_Data\\_quadrats\\{j}.shp'
    quadratsSplit[i].to_file(outPath)

In [6]:
# Clip raster using GDAL by transect
# https://gis.stackexchange.com/questions/45053/gdalwarp-cutline-along-with-shapefile

for i in range(1,4):
    poly =  f'E:\\Sync\\_Documents\\_Letter_invasives\\_Data\\_quadrats\\{i}.shp'
    ds = gdal.Warp(f"E:\\Sync\\_Documents\\_Letter_invasives\\_Data\\_quadrats\\clippedQd{i}.tif", rasPath, cropToCutline=True, cutlineDSName=poly, format="GTiff")
    ds = None # Close object

In [37]:
# Final conve

outDfs =[]
for i in range(1,4):
    tempDfs = []
    for j in range(1,9):
        path = "E:\\Sync\\_Documents\\_Letter_invasives\\_Data\\_quadrats\\clippedQd{}.tif"
        rds = rioxarray.open_rasterio(path.format(str(i))).sel(band=j)
        df = rds.to_dataframe('band'+str(j))
        df.drop(columns=['spatial_ref'], axis=1, inplace=True)
        df.reset_index(level=['x', 'y'], inplace=True, drop=False)
        df.reset_index(inplace=True, drop=True)
        tempDfs.append(df) # Conversion
    dfConcat = pd.concat([tempDfs[0]['x'], tempDfs[0]['y'], tempDfs[0]['band1'], tempDfs[1]['band2'], tempDfs[2]['band3'], tempDfs[3]['band4'], tempDfs[4]['band5'], tempDfs[5]['band6'], tempDfs[6]['band7'], tempDfs[7]['band8']], axis=1)
    dfConcat = dfConcat.loc[dfConcat['band1']>=0,:]
    outDfs.append(dfConcat)


In [38]:
outDfs = pd.concat(outDfs, axis=0, ignore_index=True)
gdf = gpd.GeoDataFrame(outDfs, crs='EPSG:26911', geometry=gpd.points_from_xy(outDfs.x, outDfs.y))

gdf.to_file("E:\\Sync\\_Documents\\_Letter_invasives\\_Data\\_quadrats\\rasters2points.shp")

### Further removal of potential IAS points was done manually in arcGIS